# Tensorflow(pb) 转 ONNX

参考: [TVM Tensorflow 前端](https://xinetzone.github.io/tvm/docs/arch/frontend/tensorflow.html)

下面以 [mobilenet_v2 float_v2_1.4_224](https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.4_224.tgz) 为例，展示 Tensorflow pb 模型转换为 ONNX 模型的过程:

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.core.framework.graph_pb2 import GraphDef

2023-06-21 09:49:59.705377: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-21 09:49:59.753895: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-21 09:49:59.754725: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 09:50:00.507851: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class MobilenetV2(tf.keras.Model):
    def __init__(self, frozen_path, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.graph_def = self._read_graph_def(frozen_path)
        self.output_names = ['output']
    
    def _read_graph_def(self, frozen_path):
        with open(frozen_path, 'rb') as f:
            graph_def = GraphDef()
            graph_def.ParseFromString(f.read())
        return graph_def
    
    @tf.function(input_signature=[tf.TensorSpec([None, 3, 224, 224], 
                                                 tf.float32, name="input")])
    def call(self, x):
        # x = tf.convert_to_tensor(x, tf.float32) # 确保输入是 tensor
        x = tf.transpose(x, perm=(0, 2, 3, 1)) # NCHW -> NHWC
        return tf.graph_util.import_graph_def(
            self.graph_def, input_map={'input:0': x}, 
            return_elements=['MobilenetV2/Predictions/Reshape_1:0']
        )[0]

In [3]:
import tf2onnx
import onnx

input_signature = [tf.TensorSpec([1, 3, 224, 224], tf.float32, name="data")]
frozen_path = '/media/pc/data/board/arria10/lxw/tests/npu_user_demos/models/mobilenet_v2_tf/weight/mobilenet_v2_1.4_224_frozen.pb'
model = MobilenetV2(frozen_path)
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13)
onnx.save(onnx_model, "temp/mobilenet_v2_tf.onnx")

2023-06-21 09:50:13.717847: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-06-21 09:50:14.762085: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'data' with dtype float and shape [1,3,224,224]
	 [[{{node data}}]]
2023-06-21 09:50:14.812947: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2023-06-21 09:50:14.813150: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-21 09:50:15.141353

测试一致性:

In [4]:
from PIL import Image
image_size = 224
path = '/media/pc/data/board/arria10/lxw/data/test/cat.png' # 将要预测的图片路径

with Image.open(path) as im:
    if im.mode != "RGB":
        im.convert("RGB")
    im = im.resize((224, 224))
    image = np.asarray(im)
image = image/128 -1
images = np.expand_dims(image, 0)
images = images.transpose((0, 3, 1, 2))

In [5]:
model = MobilenetV2(frozen_path)
tf_output = model(images)
model.summary()

Model: "mobilenet_v2_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [6]:
import set_env
import tvm
from tvm import relay
from tvm.relay.frontend import from_onnx

shape_dict = {"data": [1, 3, 224, 224]}
mod, params = from_onnx(
    onnx_model,
    shape_dict,
    freeze_params=True
)
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, "llvm", params=params)
inputs_dict = {"data": images}
mlib_proxy = tvm.contrib.graph_executor.GraphModule(lib["default"](tvm.cpu()))
mlib_proxy.run(**inputs_dict)
np.testing.assert_allclose(
    tf_output.numpy(), 
    mlib_proxy.get_output(0).numpy(),
    rtol=1e-07, atol=1e-5
)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.
